# Sberbank version of AutoML (AI Lab production) 

Данное решение представляет собой Python библиотеку, позволяющую автоматически обучать модель и проверять их качество на тестовых датасетах.

**!!!! Данная версия не имеет обратной совместимости с версиями ранее 1.3.0, комментарии см. ниже**

**V1.3.0. Обновления по сравнению с предыдущей версией:**

1. Отсутсвует аргумент model_dir при создании AutoML. При тренировке и инференсе на диск ничего не сохраняется. Исключение - если указана cahce_dir
2. Появилась возможность указаня cache_dir для train/predict. Аргумент не обязателен, но повышает эффективность обработки текстов. Эту директорию можно удалить по завершении работы, а можно оставить для ускорения повторных экспериментов на тех же данных. 
3. Появилась возможность обучать модели из памяти (передавать DataFrame) без сохранения его в csv. Интерфейс точно такой же, только вместо пути до файла можно передать другой объект
4. Появилась возможность ускорять инференс за счет параллельного выполнения. Аргумент - n_jobs
5. Появился sklearn подобный интерфейс (см. Package_usage_in_memory)
6. Для сохранения и загрузки моделей методам .save и .load теперь надо передавать путь до файла модели. Так же теперь возможно сохранение при помощи Pickle 

Еще раз отмечу здесь **важное замечание**, которое необходимо учитывать при работе с библиотекой: в случае, если используемый для тренировки датасет не помешается в заданную память, в логе будет выведено сообщение "We are sampling the data to fit the memory" - это означает, что датасет для непосредственной тренировки моделей был семплирован из исходного для того, чтобы уместиться в заданную память. 

При решении задачи регрессии можно полностью игнорировать данный факт (либо поставить больше памяти чтобы использовать больше данных для обучения).

При решении задачи классификации данный факт можно игнорировать в случае, если необходимо просто оценить метрику полученной модели (Gini, AUC и т.п.). Если планируется использовать полученные из модели вероятности в дальнейшем бизнес процессе, то **необходимо самостоятельно сделать для них калибровку**.

In [ ]:
import time
start_time = time.time()

# Package installation 

In [ ]:
!pip uninstall --yes sber_ailab_automl 
!python setup.py install --user

# Necessary imports 

In [ ]:
import sber_ailab_automl as saa
from sber_ailab_automl import SAAClassifier, SAARegressor
import joblib
import numpy as np
import pandas as pd
from sklearn.metrics import roc_auc_score, mean_squared_error
import logging
import sys
logging.basicConfig(level=logging.INFO, stream=sys.stdout,
                        format='[%(asctime)-15s] %(levelname)-8s: %(message)s')
logger = logging.getLogger(__name__)

In [ ]:
saa.__version__

# Set parameters

Опишем те параметры, которые можно передать решению для работы:

- `params['vCPULimit']` - ограничение по количеству используемых виртуальных ядер процессора
- `params['memoryLimit']` - ограничение на используемую память
- `params['timeLimit']` - ограничение на время работы
- `params['type']` - тип задачи ('classification' или 'regression')

- `params['debugFlag']` - флаг debug режима (если True, то печатается лог тренировки; если False - то нет)
- `params['featureRoles']` - параметр, в котором указываются роли и типы переменных в виде словаря:
    * `target` - строка с названием колонки целевой переменной (**обязательный параметр**)
    * `line_id` - строка с названием колонки id строки (необязательный параметр)
    * `group` - строка с названием колонки-группы для объекта, используется для GroupKFold валидации (необязательный параметр). **Важное замечание**: пока что поддерживается только одна колонка в качестве группы, а также данная колонка НЕ должна содержать пропущенных значений
    * `drop` - массив названий колонок, которые необходимо удалить из датасета (необязательный параметр)
    * `numeric` - массив названий колонок, которые являются вещественными признаками (необязательный параметр)
    * `datetime` - массив названий колонок, которые являются датами (в указанном в параметре `params['datetimeFormat']` формате) (необязательный параметр)
    * `string` - массив названий колонок, которые являются строками (категориальными признаками) (необязательный параметр)
    * `id` - массив названий колонок, которые являются id-шниками (необязательный параметр)
- `params['rowsToAnalyze']` - количество строчек, по которым автоматически определяются типы данных
- `params['datetimeFormat']` - формат, в котором в файле записаны даты (более подробно о форматах можно посмотреть [здесь](https://docs.python.org/3/library/datetime.html#strftime-strptime-behavior))
- `params['LuckySeed']` - фиксируемый seed для построения воспроизводимой кросс-валидации (также может быть использован при построении ансамбля AutoML моделей, построенных на разных CV разбиениях)
- `params['KFolds']` - количество частей, на которые разбивается выборка в процессе решения задачи
- `params['metric']` - метрика, которую необходимо оптимизировать:
    * `None` - используются дефолтные метрики AutoML (AUC для классификации и RMSE для регрессии)
    * `Func` - для кастомной метрики, которая определена функцией `Func(y_true, y_pred)`
- `params['featureSelParams']` - параметры для использования отбора признаков внутри AutoML, используется нотация tuple:
    * Первый элемент tuple отвечает за включение/отключение отбора признаков, а также за максимальное количество признаков, которые нужно использовать в модели (примеры значений: `None` - отбор признаков выключен; `-1` - набор признаков включен, количество выбираемых признаков не ограничено;  `N > 0` - отбор включен, при отборе оставить не более N признаков)
    * Второй элемент tuple отвечает за размер группы признаков, одновременно проверяемых в процессе отбора (может быть от 1 до +inf). Чем меньше этот параметр, тем более тщательно будут отбираться признаки, но тем дольше это будет происходить, поэтому необходимо выбирать trade-off скорости/результата для конкретного датасета.
    * Третий элемент tuple отвечает за отбрасывание признаков по Permutation Importance, которые ухудшают качество модели (отрицательный Permutation Importance при использовании метрики, которую нужно максимизировать, и положительный в противном случае), перед отбором признаков.

In [ ]:
params = {}
params['vCPULimit'] = 20
params['memoryLimit'] = 20 * 1024
params['timeLimit'] = 300 * 60
params['type'] = 'regression'
params['debugFlag'] = True
params['featureRoles'] = {
    'target': 'target',
    'line_id': 'line_id',
    #'group': 'group',
    'drop': [],
    'numeric': [],
    'datetime': [],
    'string': [],
    'id': [],
    'text': [],
}
params['rowsToAnalyze'] = 100000
params['datetimeFormat'] = '%Y-%m-%d'
params['LuckySeed'] = 42
params['KFolds'] = 5
params['metric'] = None 
params['featureSelParams'] = (-1, 5, True)

In [ ]:
automl = saa.AutoML(vCPULimit = params['vCPULimit'], 
                    memoryLimit = params['memoryLimit'], 
                    timeLimit = params['timeLimit'],  
                    datetime_format = params['datetimeFormat'], 
                    analyze_rows = params['rowsToAnalyze'],
                    cv_random_state = params['LuckySeed'],
                    KFolds = params['KFolds'],
                    metric = params['metric'],
                    featureSelParams = params['featureSelParams'])

train_df = pd.read_csv('example_data/check_1_r/train.csv')

res_df, feat_imp = automl.train(train_df, params['featureRoles'], params['type'], use_ids = False)
automl.save('temp_model.pkl')

# Load trained model and predict

В ячейке ниже производится объявление модели AutoML, ее загрузка и предсказание целевой переменной для тестового файла. Все необходимые параметры также задаются в ячейке с параметрами.

In [ ]:
%%time
automl = saa.AutoML(vCPULimit = params['vCPULimit'], 
                    memoryLimit = params['memoryLimit'], 
                    timeLimit = params['timeLimit'], 
                    datetime_format = params['datetimeFormat'], 
                    analyze_rows = params['rowsToAnalyze'],
                    cv_random_state = params['LuckySeed'],
                    KFolds = params['KFolds'],
                    metric = params['metric'],
                    featureSelParams = params['featureSelParams'])

automl.load('temp_model.pkl')

test_df = pd.read_csv('example_data/check_1_r/test.csv')

# файл для сохранения предсказания текста можно не передавать, тогда на диск ничего не запишется
preds_df, score, res_df_test = automl.predict(test_df, n_jobs = 4)

In [ ]:
# можно сделать предсказание без предварительного чтения, как раньше 
preds_df, score, res_df_test = automl.predict('example_data/check_1_r/test.csv', n_jobs = 1)

# SKLEARN WRAPPER

Появилась возможность пользоваться sklearn интерфейсом при обучении из пакмяти. Такая обертка работает как с датафреймами, так же с и np.array массивами без названий колонок. Различие - в featureRoles теперь отсутствует таргет, он передается на этапе обучения

In [ ]:
# при обучении из np.array в data_roles индексы фичей
model = SAARegressor(data_roles = {
    'line_id': 40,
    'drop': [],
    'numeric': [],
    'datetime': [0],
    'string': [],
    'id': [],
    'text': []
}, n_jobs = 4, KFolds = 5)
X = train_df.drop('target', axis = 1).values
y = train_df['target'].values

model.fit(X, y)
# альтернативно с датафреймом

joblib.dump(model, 'temp_model.pkl')

model1 = joblib.load('temp_model.pkl')
preds = model1.predict(X)


# Альтернативный вариант с DataFrame

In [ ]:
model = SAARegressor(data_roles = {
    'line_id': 'line_id',
    'drop': [],
    'numeric': [],
    'datetime': [],
    'string': [],
    'id': [],
    'text': []
}, n_jobs = 4, KFolds = 5)
X = train_df.drop('target', axis = 1)# .values
y = train_df['target']# .values

model.fit(X, y)
# альтернативно с датафреймом

joblib.dump(model, 'temp_model.pkl')

model1 = joblib.load('temp_model.pkl')
preds = model1.predict(X)


In [ ]:
model.fit(X, y)

# Можно создать sklearn обертку из AutoML

In [ ]:
model = SAARegressor(automl = automl)
model.predict(train_df)